In [33]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# `Logit` on Orders - Logistic Regression (~1h)

## Select features

🎯 Haydi `wait_time` ve `delay_vs_expected` değişkenlerinin çok `iyi/kötü review`lar üzerindeki etkisini inceleyelim.

👉 `orders` training_set’imizi kullanarak iki adet `multivariate logistic regression` çalıştıracağız:
- `logit_one` → `dim_is_one_star` tahmini için  
- `logit_five` → `dim_is_five_star` tahmini için.

 

In [34]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

👉 Dataset’inizi import edin:

In [35]:
from olist.order import Order
orders = Order().get_training_data(with_distance_seller_customer=True)

👉 Kullanmak istediğiniz feature’ları bir listede seçin:

⚠️ Data leakage yaratmadığınızdan emin olun (yani target’tan türetilmiş feature’ları seçmeyin)

💡 `wait_time` ve `delay_vs_expected` değişkenlerinin etkisini anlayabilmek için diğer feature’ların etkisini kontrol etmemiz gerekir, bu yüzden listenize ilgili olabilecek tüm feature’ları dahil edin.

🕵🏻 Feature’larınızın `multicollinearity` durumunu `VIF index` kullanarak kontrol edin.

* Çok yüksek olmamalıdır (tercihen < 10), böylece partial regression coefficient’larına ve ilgili `p-values` değerlerine güvenebiliriz.
* Verinizi standardize etmeyi unutmayın!
    * Bir `VIF Analysis`, bir feature’ın diğer feature’lara karşı regresyonunu yaparak hesaplanır...
    * Bu yüzden herhangi bir linear regression çalıştırmadan önce feature’ların `scale etkisini kaldırmak` ve eşit öneme sahip olmalarını sağlamak istersiniz!
    
    
📚 <a href="https://www.statisticshowto.com/variance-inflation-factor/">Statistics How To - Variance Inflation Factor</a>

📚  <a href="https://online.stat.psu.edu/stat462/node/180/">PennState - Detecting Multicollinearity Using Variance Inflation Factors</a>

In [60]:
orders.head()
features= ["wait_time", "delay_vs_expected","number_of_items", "number_of_sellers", "price", "distance_seller_customer"]


⚖️ Standardize etme:

In [65]:
#standarized
orders_standardized=orders.copy()
mu=orders[features].mean()
sigma=orders[features].std()
orders_standardized[features]= (orders[features]-mu)/sigma
orders_standardized

,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status,dim_is_five_star,dim_is_one_star,review_score,number_of_items,number_of_sellers,price,freight_value,distance_seller_customer
0,e481f51cbdc54678b7cc49136f2d6af7,-0.431192,15.544063,-0.161781,delivered,0,0,4,-0.264595,-0.112544,-0.513802,8.72,-0.979475
1,53cdb2fc8bc7dce0b6741e2150273451,0.134174,19.137766,-0.161781,delivered,0,0,4,-0.264595,-0.112544,-0.086640,22.76,0.429743
2,47770eb9100c2d0c44946d9cf07ec65d,-0.329907,26.639711,-0.161781,delivered,1,0,5,-0.264595,-0.112544,0.111748,19.22,-0.145495
3,949d5b44dbf5de918fe9c16f97b45f8a,0.073540,26.188819,-0.161781,delivered,1,0,5,-0.264595,-0.112544,-0.441525,27.20,2.054621
4,ad21c59c0840e6cb83a9ceb5573f8159,-1.019535,12.112049,-0.161781,delivered,1,0,5,-0.264595,-0.112544,-0.562388,8.72,-0.959115
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95875,9c5dedf39a927c1b2549525ed64a053c,-0.454309,18.587442,-0.161781,delivered,1,0,5,-0.264595,-0.112544,-0.311513,13.08,-0.893033
95876,63943bddc261676b46f01ca7ac2f7bd8,1.023841,23.459051,-0.161781,delivered,0,0,4,-0.264595,-0.112544,0.183977,20.10,-0.212797
95877,83c1379a015df1e13d02aae0204711ab,1.305780,30.384225,-0.161781,delivered,1,0,5,-0.264595,-0.112544,0.333684,65.02,0.617630
95878,11c177c8e97725db2631073c19f07b62,0.483664,37.105243,-0.161781,delivered,0,0,2,1.601605,-0.112544,1.075186,81.18,-0.387558


👉 Olası multicollinearity durumlarını analiz etmek için VIF Analysis’inizi çalıştırın:

In [67]:
X_standardized = orders_standardized[features]
X_standardized.head()

,wait_time,delay_vs_expected,number_of_items,number_of_sellers,price,distance_seller_customer
0,-0.431192,-0.161781,-0.264595,-0.112544,-0.513802,-0.979475
1,0.134174,-0.161781,-0.264595,-0.112544,-0.086640,0.429743
2,-0.329907,-0.161781,-0.264595,-0.112544,0.111748,-0.145495
3,0.073540,-0.161781,-0.264595,-0.112544,-0.441525,2.054621
4,-1.019535,-0.161781,-0.264595,-0.112544,-0.562388,-0.959115


In [69]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
df=pd.DataFrame()
df["features"]=X_standardized.columns
df["vif_index"]=[vif(X_standardized.values, i) for i in range(X_standardized.shape[1])]
round(df,2)


,features,vif_index
0,wait_time,2.61
1,delay_vs_expected,2.21
2,number_of_items,1.12
3,number_of_sellers,1.09
4,price,1.03
5,distance_seller_customer,1.33


## Logistic Regressions

👉 İki adet `Logistic Regression` modeli fit edin:
- `logit_one` → `dim_is_one_star` tahmini için
- `logit_five` → `dim_is_five_star` tahmini için.

`Logit 1️⃣`

In [70]:
logit_one=smf.logit(formula="dim_is_one_star ~"+"+".join(features), data=orders_standardized).fit()
logit_one.summary()


Optimization terminated successfully.
         Current function value: 0.273598
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:        dim_is_one_star   No. Observations:                95872
Model:                          Logit   Df Residuals:                    95865
Method:                           MLE   Df Model:                            6
Date:                Thu, 22 Jan 2026   Pseudo R-squ.:                  0.1447
Time:                        00:16:55   Log-Likelihood:                -26230.
converged:                       True   LL-Null:                       -30669.
Covariance Type:            nonrobust   LLR p-value:                     0.000
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -2.4673      0.013   -190.880      0.000      -2.493      -2.442
wait_time                    0.7101      0.017     42.178      0.000       0.677       0.743
delay_vs_expected            0.2570      0.018     13.910      0.000       0.221       0.293
number_of_items              0.2409      0.009     26.272      0.000       0.223       0.259
number_of_sellers            0.1776      0.008     22.621      0.000       0.162       0.193
price                        0.0431      0.010      4.105      0.000       0.023       0.064
distance_seller_customer    -0.1844      0.013    -13.787      0.000      -0.211      -0.158
============================================================================================
"""

`Logit 5️⃣`

In [71]:
logit_five=smf.logit(formula="dim_is_five_star ~"+"+".join(features), data=orders_standardized).fit()
logit_five.summary()

Optimization terminated successfully.
         Current function value: 0.636832
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:       dim_is_five_star   No. Observations:                95872
Model:                          Logit   Df Residuals:                    95865
Method:                           MLE   Df Model:                            6
Date:                Thu, 22 Jan 2026   Pseudo R-squ.:                 0.05805
Time:                        00:18:19   Log-Likelihood:                -61054.
converged:                       True   LL-Null:                       -64817.
Covariance Type:            nonrobust   LLR p-value:                     0.000
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    0.3388      0.007     47.337      0.000       0.325       0.353
wait_time                   -0.5214      0.012    -44.772      0.000      -0.544      -0.499
delay_vs_expected           -0.4335      0.024    -18.442      0.000      -0.480      -0.387
number_of_items             -0.1344      0.008    -17.742      0.000      -0.149      -0.120
number_of_sellers           -0.1426      0.008    -18.212      0.000      -0.158      -0.127
price                        0.0225      0.007      3.180      0.001       0.009       0.036
distance_seller_customer     0.0886      0.008     11.126      0.000       0.073       0.104
============================================================================================
"""

💡 Şimdi bu iki logistic regression’ın sonuçlarını analiz etme zamanı:

- Partial coefficient’ları kendi kelimelerinizle yorumlayın.
- `p-values` kullanarak istatistiksel anlamlılıklarını kontrol edin.
- Coefficient önemleri açısından `logit_one` ve `logit_five` arasında herhangi bir fark görüyor musunuz?

In [73]:
# Aşağıdaki cümlelerden doğru olanları aşağıdaki listeye kaydedin.

a = "delay_vs_expected influences five_star ratings even more than one_star ratings"
b = "wait_time influences five_star ratings even more than one_star"

your_answer = [a]

🧪 __Kodunu Test Et__

In [74]:
from nbresult import ChallengeResult

result = ChallengeResult('logit',
    answers = your_answer
)
result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /Users/senemyilmaz/.pyenv/versions/workintech/bin/python
cachedir: .pytest_cache
rootdir: /Users/senemyilmaz/data-logit/tests
plugins: anyio-4.8.0, typeguard-4.4.2
collecting ... collected 1 item

test_logit.py::TestLogit::test_question PASSED                           [100%]

============================== 1 passed in 0.01s ===============================


💯 You can commit your code:

git add tests/logit.pickle

git commit -m 'Completed logit step'

git push origin master



<details>
    <summary>- <i>Açıklamalar ve ileri seviye kavramlar</i> -</summary>


> _Diğer tüm şeyler sabitken, `delay factor`, 1-yıldız review alma ihtimalini etkilemesinden bile daha fazla, 5-yıldızdan mahrum kalma ihtimalini artırma eğilimindedir. Muhtemelen bunun sebebi, 1-yıldız review’ların bizzat çok kötü ürünleri hedeflemesi, kötü teslimatları değil._

❗️ Ancak tamamen titiz olmak için, **iki farklı modelin coefficient’larını karşılaştırırken daha dikkatli olmamız gerekir**, çünkü **benzer popülasyonlara dayanmayabilirler**!
    Burada 2 alt popülasyonumuz var: (1-yıldız verenler ve 5-yıldız verenler) ve bunlar doğaları gereği farklı davranış kalıpları sergileyebilirler. 5-yıldız vermeye daha meyilli “mutlu insanlar”ın, “gecikme” veya “fiyat” söz konusu olduğunda, 1-yıldızı “Lucky-Luke gibi ateşleyen” “huysuz insanlara” göre daha az hassas olmaları gayet mümkün...

</details>



🏁 Tebrikler!

💾 `logit.ipynb` notebook’unuzu commit ve push etmeyi unutmayın!